### Exercício 4

A empresa americana KMX do setor automobilístico lançará três novos modelos de carros no próximo ano: modelo Arlington, modelo Marilandy e modelo Gristedes. A produção de cada um dos modelos passa pelos seguintes processos: injeção, fundição, usinagem, estamparia e acabamento. Os tempos médios de operação (minutos) de uma unidade de cada componente encontram-se na Tabela 2.15. Cada uma das operações é 100% automatizada. A quantidade de máquinas disponíveis para cada setor também se encontra na mesma tabela. É importante mencionar que cada máquina trabalha 16 horas por dia, de segunda a sexta-feira. O lucro unitário, além do potencial mínimo de vendas por semana, de cada modelo de automóvel, de acordo com o setor comercial, está especificado na Tabela 2.16. Supondo que 100% dos modelos serão vendidos, formule o problema de programação linear que busca determinar as quantidades de automóveis de cada modelo a serem fabricados, a fim de maximizar o lucro líquido semanal.

#### Tabela 2.15 Tempo médio de operação (minutos) de uma unidade de cada componente e total de máquinas disponíveis

| Setor      | Arlington | Marilandy | Gristedes | Máquinas disponíveis |
|------------|-----------|-----------|-----------|----------------------|
| Injeção    | 3         | 5         | 4         | 6                    |
| Fundição   | 5         | 4         | 3         | 4                    |
| Usinagem   | 2         | 5         | 4         | 8                    |
| Estamparia | 4         | 3         | 5         | 3                    |
| Acabamento | 2         | 4         | 3         | 5                    |

#### Tabela 2.16 Lucro unitário e potencial mínimo de vendas semanal por produto

| Modelos   | Lucro unitário (US$) | Potencial mín. vendas (unidades/semana) |
|-----------|----------------------|-----------------------------------------|
| Arlington | 2.500                | 50                                      |
| Marilandy | 3.000                | 20                                      |
| Gristedes | 2.800                | 30                                      |


###  Modelo Matemático

### Modelo Formal do Problema

#### Índices:
- \( i \): Modelos de automóveis, onde \( i \in \{ \text{Arlington}, \text{Marilandy}, \text{Gristedes} \} \).
- \( j \): Setores de produção, onde \( j \in \{ \text{Injeção}, \text{Fundição}, \text{Usinagem}, \text{Estamparia}, \text{Acabamento} \} \).

#### Parâmetros:
- \( t_{ij} \): Tempo médio de operação (em minutos) para produzir uma unidade do modelo \( i \) no setor \( j \).
- \( m_j \): Número de máquinas disponíveis no setor \( j \).
- \( h \): Horas de trabalho por dia (16 horas).
- \( d \): Dias de trabalho por semana (5 dias).
- \( L_i \): Lucro unitário (em dólares) do modelo \( i \).
- \( S_i \): Potencial mínimo de vendas semanais do modelo \( i \).

#### Variáveis de Decisão:
- \( x_i \): Quantidade de automóveis do modelo \( i \) a serem fabricados por semana.

#### Função Objetivo:
Maximizar o lucro líquido semanal:

$$
Z = \sum_{i} L_i \cdot x_i
$$

#### Restrições:

1. **Disponibilidade de máquinas (tempo em minutos):**

$$
\sum_{i} t_{ij} \cdot x_i \leq m_j \cdot h \cdot 60 \cdot d, \quad \forall j
$$

2. **Potencial mínimo de vendas:**

$$
x_i \geq S_i, \quad \forall i
$$

3. **Não negatividade:**

$$
x_i \geq 0, \quad \forall i
$$


In [ ]:
import pyomo.environ as pyo

In [2]:
# Criação do modelo
model = pyo.ConcreteModel()

In [3]:

# Conjuntos
model.P = pyo.Set(initialize=['Arlington', 'Marilandy', 'Gristedes'])  # Modelos de carros
model.S = pyo.Set(initialize=['Injecao', 'Fundicao', 'Usinagem', 'Estamparia', 'Acabamento'])  # Setores

In [4]:
# ------------------------------
# Parâmetros
# ------------------------------
lucro = {'Arlington': 2500, 'Marilandy': 3000, 'Gristedes': 2800}
model.lucro = pyo.Param(model.P,initialize=lucro)

demanda_min = {'Arlington': 50, 'Marilandy': 30, 'Gristedes': 30}
model.demanda_min = pyo.Param(model.P,initialize=demanda_min)

tempo_operacao = {
    ('Injecao', 'Arlington'): 3, ('Injecao', 'Marilandy'): 4, ('Injecao', 'Gristedes'): 3,
    ('Fundicao', 'Arlington'): 5, ('Fundicao', 'Marilandy'): 5, ('Fundicao', 'Gristedes'): 4,
    ('Usinagem', 'Arlington'): 2, ('Usinagem', 'Marilandy'): 4, ('Usinagem', 'Gristedes'): 4,
    ('Estamparia', 'Arlington'): 4, ('Estamparia', 'Marilandy'): 5, ('Estamparia', 'Gristedes'): 5,
    ('Acabamento', 'Arlington'): 2, ('Acabamento', 'Marilandy'): 3, ('Acabamento', 'Gristedes'): 3
}
model.tempo_operacao = pyo.Param(model.S, model.P, initialize=tempo_operacao)

maquinas_disponiveis = {'Injecao': 6, 'Fundicao': 8, 'Usinagem': 5, 'Estamparia': 8, 'Acabamento': 5}
model.maquinas_disponiveis = pyo.Param(model.S, initialize=maquinas_disponiveis)

tempo_semanal_por_maquina = 4800  # minutos por semana
model.tempo_semanal_por_maquina = pyo.Param(initialize=tempo_semanal_por_maquina)


In [5]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------
model.x = pyo.Var(model.P, domain=pyo.NonNegativeReals)  # Quantidade produzida de cada modelo


In [6]:

# ------------------------------
# Função Objetivo
# ------------------------------
def lucro_total(model):
    return sum(lucro[j] * model.x[j] for j in model.P)
model.objetivo = pyo.Objective(rule=lucro_total, sense=pyo.maximize)


In [7]:
# ------------------------------
# Restrições
# ------------------------------


# 1. Restrições de capacidade por setor
def restricao_capacidade(model, i):
    return sum(tempo_operacao[i, j] * model.x[j] for j in model.P) <= maquinas_disponiveis[i] * tempo_semanal_por_maquina
model.capacidade = pyo.Constraint(model.S, rule=restricao_capacidade)

# 2. Restrições de demanda mínima
def restricao_demanda(model, j):
    return model.x[j] >= demanda_min[j]
model.demanda = pyo.Constraint(model.P, rule=restricao_demanda)

In [8]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex4.lp", io_options={"symbolic_solver_labels": True})

('ex4.lp', 2304139912576)

In [9]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)



LP   has 8 rows; 3 cols; 18 nonzeros
Coefficient ranges:
  Matrix [1e+00, 5e+00]
  Cost   [2e+03, 3e+03]
  Bound  [0e+00, 0e+00]
  RHS    [3e+01, 4e+04]
Presolving model
5 rows, 3 cols, 15 nonzeros  0s
5 rows, 3 cols, 15 nonzeros  0s
Presolve : Reductions: rows 5(-3); columns 3(-0); elements 15(-3)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -8.2999936572e+03 Ph1: 5(56); Du: 3(8299.99) 0s
          2    -2.2075000000e+07 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 2
Objective value     :  2.2075000000e+07
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00

Status do solver: ok
Condição de terminação: optimal
Valor da Função Objetivo: R$ 22.075.000,00


In [10]:
for i in model.P:    
    val = pyo.value(model.x[i])
    if val is not None and val > 1e-6:
        print(f"  Quantidade do produto {i} : {val:.2f}")

  Quantidade do produto Arlington : 4790.00
  Quantidade do produto Marilandy : 30.00
  Quantidade do produto Gristedes : 3575.00
